# Build the NN

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1、使用cuda

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} devices'.format(device))

Using cuda devices


## 2、定义类

我们通过子类化定义神经网络nn.Module，并在中初始化神经网络层__init__。每个nn.Module子类都对forward方法中的输入数据执行操作。

#### 常用的网络层：

- 全连接层

  ```
  Linear(in_features, out_features, bias=True)
  ```

  - in_features: 输入的特征数
  - out_features: 输出的特征数

- 随机丢弃层

  ```
  Dropout(p=0.5, inplace=False)
  ```

  - p: float, 特征丢弃的比例

- **展平层** - `Flatten()`

- 2D卷积层

  ```
  Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, ...)
  ```

  - in_channels: 输入的特征数
  - out_channels: 输出特征数 (即卷积中输出滤波器的数量）
  - kernel_size: 卷积窗口的长度
  - stride: 指定卷积的步长
  - padding: 填充数量
  
  
  
经过卷积后的图像尺寸计算公式：N = (W − F + 2P )/S+1

w为原始图像大小、F为卷积核尺寸、P为padding大小、S是步长

28 - 3 + 2 = 27; 27/3 = 9 ; 9+1 =10
```
input_image = torch.rand(1, 1, 28, 28)
print(input_image.size())
conv1 = nn.Conv2d(1, 16, 3, stride=3, padding=1)
conv1_img = conv1(input_image)
print(conv1_img.size())
```
反卷积：output = (input-1)stride+outputpadding -2padding+kernelsize


- 最大池化层

  ```
  MaxPooling2D(kernel_size, stride=None, padding=0, ...)
  ```

  - kernel_size: 池化窗口的长度
  - stride: 指定池化的步长
  - padding: 填充数量

- 平均池化层

  ```
  AvgPool2d((kernel_size, stride=None, padding=0, ...)
  ```

  - kernel_size: 池化窗口的长度
  - stride: 指定池化的步长
  - padding: 填充数量

- 自适应池化

  ```
  AdaptiveMaxPool2d(output_size, ...), AdaptiveAvgPool2d(output_size, ...)
  ```

  - output_size: 特征输出大小(H, W)
  - 对于任何输入大小，输出大小均为HxW。输出特征的数量等于输入特征的数量

- **非线性激活** - `ReLU(), Sigmoid()`

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [5]:
# 第三方库 torchsummary 打印模型详情
# from torchsummary import summary

# summary(model, (1, 28, 28),)

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


## 3、Model Layers
   细分FashionMNIST模型中的各个层。

   为了说明这一点，我们将抽取3个大小为28x28的图像的小批量样本，并观察当它通过网络传递时发生了什么。

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten：

我们初始化nn.Flatten 层，以将每个2D 28x28图像转换为784个像素值的连续数组（维持了最小批处理尺寸（在dim = 0时））。

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear
The linear layer is a module that applies a linear transformation on the input using it’s stored weights and biases.
使用它的存储的权重和偏置输入施加线性变换的模块

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
非线性激活会在模型的输入和输出之间创建复杂的映射。They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.5681, -0.0246, -0.3696,  0.0363,  0.0845, -0.0011, -0.6233, -0.3214,
          0.0831, -0.1628, -0.5470, -0.2869,  0.1306,  0.1936, -0.2363,  0.7477,
         -0.0072,  0.7228, -0.2375,  0.1610],
        [ 0.5125,  0.2423, -0.3006,  0.2512,  0.1474,  0.5515, -0.8556, -0.3920,
          0.3432,  0.0814, -0.5375, -0.6362,  0.1519,  0.2941,  0.0228,  0.6906,
         -0.3687,  0.7015, -0.3500,  0.1392],
        [ 0.5871,  0.3924, -0.0084, -0.1620, -0.0745, -0.0149, -0.7710, -0.2393,
         -0.2255, -0.0171, -0.5344, -0.3291,  0.1266, -0.0139, -0.7775,  0.8234,
         -0.2403,  0.4986, -0.1676,  0.5126]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.5681, 0.0000, 0.0000, 0.0363, 0.0845, 0.0000, 0.0000, 0.0000, 0.0831,
         0.0000, 0.0000, 0.0000, 0.1306, 0.1936, 0.0000, 0.7477, 0.0000, 0.7228,
         0.0000, 0.1610],
        [0.5125, 0.2423, 0.0000, 0.2512, 0.1474, 0.5515, 0.0000, 0.0000, 0.3432,
         0.0814, 0.0000, 0.0000, 0.1519, 0.2941, 0.022

#### nn.Sequential
nn.Sequential是模块的有序容器。

数据按照定义的顺序通过所有模块。您可以使用顺序容器将类似`seq_modules`的快速网络组合在一起

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax
神经网络的最后一个线性层。

将对数缩放为[0，1]值，该值表示每个类别的模型的预测概率。

dim参数表示必须将值加起来为1的维度。

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 4、模型参数
神经网络内部的许多层均已参数化，即具有相关的权重和偏差，这些权重和偏差在训练期间进行了优化。

子类化nn.Module自动跟踪模型对象内部定义的所有字段，并使用模型parameters()或named_parameters()方法访问所有参数。

In [15]:
# 遍历每个参数，并打印其大小和值的预览
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer : {name} | Size: {param.size()} | Value : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer : linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Value : tensor([[-0.0287,  0.0167, -0.0114,  ...,  0.0278, -0.0040,  0.0289],
        [-0.0075, -0.0061,  0.0009,  ..., -0.0065,  0.0088, -0.0240]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.0.bias | Size: torch.Size([512]) | Value : tensor([-0.0020, -0.0142], device='cuda:0', grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Value : tensor([[-0.0310,  0.0048,  0.0398,  ..., -0.0411, -0.0377, -0.0240],
        [ 0.0399,  0.0354, -0.0396,  ..., -0.0335, -0.0266,  0.0406]],
       device='cuda